In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
def load_images_from_folder(folder, image_size=(128, 128)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                img_path = os.path.join(label_folder, filename)
                try:
                    img = Image.open(img_path).convert('RGB')
                    img = img.resize(image_size)
                    img_array = np.array(img).flatten()  # Flatten the image
                    images.append(img_array)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)


In [ ]:
def count_classes(folder_path):
    return len([name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))])


In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, precision, recall, f1



In [ ]:
train_folder_path = '/content/drive/My Drive/dataset/train'
test_folder_path = '/content/drive/My Drive/dataset/test'
X_train, y_train = load_images_from_folder(train_folder_path, image_size=(128, 128))
X_test, y_test = load_images_from_folder(test_folder_path, image_size=(128, 128))


In [ ]:
!pip install scikit-learn
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Standardize the data (recommended for certain algorithms)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Define models
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier(n_neighbors=1),
    "Logistic Regression": LogisticRegression(max_iter=500),  # Increased max_iter
    "SVM": SVC(),
    "ANN": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=100)  # Adjust ANN parameters
}


In [ ]:
# Evaluate each model
results = []

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train_encoded)  # Use scaled data
    y_pred = model.predict(X_test_scaled)  # Ensure predictions are class labels
    accuracy = accuracy_score(y_test_encoded, y_pred)
    precision = precision_score(y_test_encoded, y_pred, average='weighted')
    recall = recall_score(y_test_encoded, y_pred, average='weighted')
    f1 = f1_score(y_test_encoded, y_pred, average='weighted')
    results.append([model_name, accuracy, precision, recall, f1])




In [ ]:
# Convert the metrics to percentages
results_percentage = []
for result in results:
    algorithm = result[0]
    accuracy = result[1] * 100
    precision = result[2] * 100
    recall = result[3] * 100
    f1_score = result[4] * 100
    results_percentage.append([algorithm, accuracy, precision, recall, f1_score])



In [ ]:
# Display results in a table format
import pandas as pd
results_df = pd.DataFrame(results_percentage, columns=["Algorithm", "Accuracy (%)", "Precision (%)", "Recall (%)", "F1 Score (%)"])
print(results_df)

             Algorithm  Accuracy (%)  Precision (%)  Recall (%)  F1 Score (%)
0        Decision Tree     43.902439      40.548416   43.902439     40.219504
1                  KNN     54.878049      48.177740   54.878049     49.474619
2  Logistic Regression     47.560976      46.510400   47.560976     45.816462
3                  SVM     47.560976      42.978616   47.560976     44.277503
4                  ANN     51.219512      47.856615   51.219512     47.882289
